# Đồ Án Cuối Kỳ


Thành viên 1:

    Họ tên: ...

    MSSV: ...


Thành viên 2:

    Họ tên: ...

    MSSV: ...
    

## Môi trường code

In [1]:
import sys
sys.executable

## Import các thư viện cần thiết

In [2]:
import pandas as pd

## Thu thập dữ liệu

Trong bài này, ta sẽ dùng dữ liệu đã được thu thập sẵn là file "data.csv" đính kèm; đây là dữ liệu về thông tin của một sản phẩm sau khi được đặt từ 1 trang web bán hàng . Dữ liệu này được lấy từ các nhóm tại PromptCloud và DataStock [ở đây](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2).

## Khám phá dữ liệu

## Đọc dữ liệu từ file csv

In [3]:
product_df = pd.read_csv('data.csv')
product_df.head()

# df.to_csv('out1.csv', sep='\t', encoding='utf-8')

,Uniq Id,Crawl Timestamp,Category,Product Title,Product Description,Brand,Pack Size Or Quantity,Mrp,Price,Site Name,Offers,Combo Offers,Stock Availibility,Product Asin,Image Urls
0,eb49cc038190f6f03c272f79fbbce894,2019-10-30 11:38:11 +0000,Skin Care,Lee posh Lactic Acid 60% Anti ageing Pigmenta...,PROFESSIONAL GRADE Face Peel: this peel stimul...,Lee Posh,NaN,2000.00,799.00,Amazon In,60.05%,NaN,YES,B072BGHNJ1,https://images-na.ssl-images-amazon.com/images...
1,1657cc30c438affede6a5060d6847363,2019-10-31 15:46:54 +0000,Skin Care,Branded SLB Works New 1.5mm Titanium 1200 nee...,Item name: 1.5mm titanium 1200 needles microne...,SLB Works,NaN,2040.00,2040.00,Amazon In,0%,NaN,YES,B07QDTZYSJ,https://images-na.ssl-images-amazon.com/images...
2,41654633cce38c8650690f6dbac01fd3,2019-10-30 09:53:23 +0000,Skin Care,Generic 1 Pc brand snail eye cream remove dar...,"Use: eye, item type: cream, net wt: 20g, gzzz:...",Generic,NaN,1824.00,1042.00,Amazon In,42.87%,NaN,YES,B07DCSN8MP,https://images-na.ssl-images-amazon.com/images...
3,08b1bd85c3efc2d7aa556fd79b073382,2019-10-29 16:16:52 +0000,Skin Care,Generic Anti Snoring Snore Stopper Sleep Apne...,Prevent the tongue from dropping backward or b...,Generic,NaN,2185.00,1399.00,Amazon In,35.97%,NaN,YES,B07GLW9VQN,https://images-na.ssl-images-amazon.com/images...
4,3ac3f213732512d1d11bb73ab3b1900f,2019-10-31 09:32:06 +0000,Grocery & Gourmet Foods,Harveys Crunchy & Creame Gourmet Delicacies C...,Harvey's wafer Cream Wafer 110g. Made in India,Harveys,NaN,594.00,570.00,Amazon In,4.04%,NaN,YES,B07NFYYLF1,https://images-na.ssl-images-amazon.com/images...


## Bỏ Các Dòng Không Phải Thông Tin Product

In [4]:
product_df = product_df.dropna(how='all', subset=['Crawl Timestamp'], axis=0)

### Dữ liệu có bao nhiêu dòng và cột?

In [5]:
num_rows = len(product_df.values)
num_cols = len(product_df.loc[0])

print(num_cols,num_rows)

15 30000


### Ý nghĩa của mỗi dòng. Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả dữ liệu từ các nhóm tại PromptCloud và DataStock và theo quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin của một một sản phẩm đang được trưng bày bán. Có vẻ không có dòng nào bị "lạc loài".

### Dữ liệu có các dòng bị lặp không?

In [6]:
have_duplicated_rows = product_df["Uniq Id"].duplicated().any()
if have_duplicated_rows:
    print('Có')
else :
    print ('Không')

Không


### Ý nghĩa của mỗi cột

Dưới đây là phần mô tả dữ liệu từ các nhóm tại PromptCloud và DataStock về các cột trong file "data.csv":
- **Uniq Id**: a unique identifier for each product.
- **Crawl Timestamp**: Data collection time for this product.
- **Category**: Product category.
- **Product Title**: where you list your product in an eCommerce store.
- **Product Description**: Detailed description of the product
- **Brand**: The brand that owns the Product.
- **Pack Size Or Quantity**: Size and quantity of products available.
- **Mrp**: .
- **Price**: price of this product.
- **Site Name**: the name of the website that is selling the product.
- **Offers**: Percentage on offer
- **Combo Offers**: combo packages.
- **Stock Availibility**: the product is available or not (if the product is available, it is YES, if it is not, it is NO) .
- **Product Asin**: ASIN (Amazon Standard Identification Number) is one of the unique product identifiers used for managing the Amazon product catalog. In a technical sense, it's a 10-digit code made up of numbers and letters.
- **Image Urls**: Link to the product's profile picture.

### Mỗi cột có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Lấy `dtype` (kiểu dữ liệu của mỗi phần tử) của mỗi cột trong `df` và lưu kết quả vào series `col_dtypes`; series này có index là tên cột. 


In [7]:
col_dtypes = product_df.dtypes
col_dtypes
product_df.head(5)


def check_dtype_element(s):
    return type(s)
def open_object_dtype(s):
    dtypes = set(s.apply(check_dtype_element))
    return dtypes

In [8]:
open_object_dtype(product_df["Image Urls"])

{str}

### Cột có dtype là object nghĩa là sao?

Ta cần biết tập các kiểu dữ liệu thật sự có trong mỗi cột để dễ dàng tiền xử lý

`product_df` có 4 cột có `dtype` là `object`: "Date", "LocationDescription", "District", "CommunityArea". Trong Pandas, kiểu dữ liệu `object` thường ám chỉ chuỗi, nhưng thật ra kiểu dữ liệu `object` có thể chứa một đối tượng bất kỳ trong Python (vì thật ra ở bên dưới kiểu dữ liệu `object` chứa địa chỉ). Nếu một cột trong dataframe có `dtype` là `object` thì có thể các phần tử trong cột này sẽ có kiểu dữ liệu khác nhau; để biết được kiểu dữ liệu thật sự của các phần tử trong cột này thì ta phải truy xuất vào từng phần tử và dùng câu lệnh `type` để xem kiểu dữ liệu, ví dụ `type(s.iloc[i])` với `s` là một cột. Lúc đọc dữ liệu từ file "mvt.csv" lên, nếu bạn để ý thì sẽ thấy warning là: "Columns (6,7) have mixed types", nghĩa là cột 6 ("District") và 7 ("CommunityArea") có `dtype` không đồng nhất (trong nội bộ mỗi cột). Ta muốn xem thử trong nội bộ mỗi cột này có các kiểu dữ liệu nào.

## Tiền xử lý (nếu cần)

Ta sẽ tiền xử lý các cột `Crawl Timestamp`,`Mrp`, `Price`, `Offers`

In [10]:
product_df["Mrp"] =  product_df["Mrp"].str.lstrip('.').astype(float)
product_df["Crawl Timestamp"] = product_df["Crawl Timestamp"].astype('datetime64[s]')
product_df["Price"] =  product_df["Price"].str.lstrip('.').astype(float)
product_df["Offers"] = product_df["Offers"].str.replace('%','').astype(float) / 100

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột numeric, các giá trị được phân phối như thế nào?

Với mỗi cột numeric bạn sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), min, max. Bạn lưu kết quả vào dataframe nume_col_profiles_df. Dataframe này có 3 dòng là "missing_ratio", "min", "max"; và có n cột (n là số cột numeric)

In [14]:
# YOUR CODE HERE
nume_col_profiles_df = pd.Series([])

def missing_ratio(s):
    return s.isna().mean() * 100

nume_col_profiles_df = pd.concat([product_df['Mrp'],product_df['Price'],product_df['Crawl Timestamp'],\
                                  product_df['Offers']], axis=1)
nume_col_profiles_df = nume_col_profiles_df.agg([missing_ratio, pd.Series.min, pd.Series.max])

/home/nson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [15]:
nume_col_profiles_df

,Mrp,Price,Crawl Timestamp,Offers
missing_ratio,2.33,2.0,0,1.553333
min,12.00,12.0,2019-10-28 14:30:19,0.000000
max,80000.00,21990.0,2019-10-31 19:13:19,0.972400


### Nhận xét về "missing_ratio", "min", "max" của các cột numeric

missing_ratio và Min và max của các cột có vẻ không có gì bất thường.

### Với mỗi cột categorical, các giá trị được phân phối như thế nào?

Với mỗi cột categorical, bạn tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng giá trị khác nhau (không xét giá trị thiếu), list/array các giá trị khác nhau (không xét giá trị thiếu). Bạn lưu kết quả vào dataframe cate_col_profiles_df. Dataframe này có 3 dòng là "missing_ratio", "num_diff_vals", "diff_vals"; và có n cột là (n là số lượng cột categorical)

In [22]:
# YOUR CODE HERE
cate_col_profiles_df = pd.Series([])


def num_diff_vals(s):
    return pd.Series.nunique(s.dropna())

def diff_vals(s):
    s=s.dropna()
    return pd.Series.unique(s.dropna())


def missing_ratio(s):
    return s.isna().mean() * 100

cate_col_profiles_df = pd.concat([product_df['Uniq Id'],product_df['Category'],product_df['Product Title'],\
                                  product_df['Product Description'],product_df['Brand'],product_df['Site Name'],\
                                  product_df['Combo Offers'],product_df['Stock Availibility'],product_df['Product Asin'],\
                                 product_df['Image Urls'],product_df['Category'],product_df['Category']], axis=1)
cate_col_profiles_df = cate_col_profiles_df.agg([missing_ratio,num_diff_vals,diff_vals ])

/home/nson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [23]:

product_df.info()
cate_col_profiles_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Uniq Id                30000 non-null  object        
 1   Crawl Timestamp        30000 non-null  datetime64[ns]
 2   Category               30000 non-null  object        
 3   Product Title          30000 non-null  object        
 4   Product Description    28010 non-null  object        
 5   Brand                  29913 non-null  object        
 6   Pack Size Or Quantity  10224 non-null  object        
 7   Mrp                    29301 non-null  float64       
 8   Price                  29400 non-null  float64       
 9   Site Name              30000 non-null  object        
 10  Offers                 29534 non-null  float64       
 11  Combo Offers           37 non-null     object        
 12  Stock Availibility     30000 non-null  object        
 13  P

,Uniq Id,Category,Product Title,Product Description,Brand,Site Name,Combo Offers,Stock Availibility,Product Asin,Image Urls,Category,Category
missing_ratio,0,0,0,6.63333,0.29,0,99.8767,0,0,0,0,0
num_diff_vals,30000,6,29630,22584,8454,1,36,2,30000,29578,6,6
diff_vals,"[eb49cc038190f6f03c272f79fbbce894, 1657cc30c43...","[Skin Care, Grocery & Gourmet Foods, Bath & Sh...",[ Lee posh Lactic Acid 60% Anti ageing Pigment...,[PROFESSIONAL GRADE Face Peel: this peel stimu...,"[Lee Posh, SLB Works, Generic, Harveys, ShiKai...",[Amazon In],[ Myoc Neem Face Wash: Buy 2 Neem face Wash 10...,"[YES, NO]","[B072BGHNJ1, B07QDTZYSJ, B07DCSN8MP, B07GLW9VQ...",[https://images-na.ssl-images-amazon.com/image...,"[Skin Care, Grocery & Gourmet Foods, Bath & Sh...","[Skin Care, Grocery & Gourmet Foods, Bath & Sh..."


### Nhận xét về "missing_ratio", "num_diff_vals", "diff_vals" của các cột categorical

## Đưa ra câu hỏi có ý nghĩa cần trả lời

## Tiền xử lý (nếu cần) + phân tích dữ liệu để trả lời cho câu hỏi 1

## Tiền xử lý (nếu cần) + phân tích dữ liệu để trả lời cho câu hỏi 2

In [24]:
!git add .
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .~lock.data.csv#
	new file:   .~lock.out1.csv#
	modified:   Main Flow.ipynb
	new file:   out1.csv



In [26]:
!git branch Son

In [28]:
!git commit -m"Tiền xử lý"

[main 9ee766b] Tiền xử lý
 4 files changed, 39171 insertions(+), 86 deletions(-)
 create mode 100644 .~lock.data.csv#
 create mode 100644 .~lock.out1.csv#
 create mode 100644 out1.csv


In [ ]:
!git push origin Son

Username for 'https://github.com': 

In [ ]:
!nguyenson-255